In [2]:
from supabase import create_client, Client
import os
from rich import print as rprint

# Supabase 클라이언트 생성
supabase: Client = create_client(
    os.environ["SUPABASE_URL"],
    os.environ["SUPABASE_KEY"]
)

print("✅ Supabase 클라이언트 초기화 완료!")
# 연결 테스트
try:
    result = supabase.table("influencers").select("*").limit(1).execute()
    print("✅ Supabase 연결 성공!")
    print(f"   테이블 접근 확인: influencers")
except Exception as e:
    print(f"❌ 연결 실패: {e}")
    print("   → 테이블을 먼저 생성했는지 확인하세요!")

✅ Supabase 클라이언트 초기화 완료!
✅ Supabase 연결 성공!
   테이블 접근 확인: influencers


- 테이블 조회는 안되네, 직접 보고 해야함.

In [13]:
response = (
    supabase.table("influencers")
    .select("*") # *로 먼저 확인한 후 요소별로 조회 ㅇㅇ
    .execute()
)
rprint(response)

APIResponse(
    data=[
        {
            'id': '79df0f54-be6e-4c7c-a742-8aa9d1131abf',
            'platform': 'instagram',
            'platform_id': 'kom_universe',
            'platform_name': '콤',
            'results_limit': 200,
            'memo': None,
            'created_at': '2025-12-22T09:00:45.154489+00:00'
        }
    ],
    count=None
)

In [19]:
# response = (
#     supabase.table("reel_metrics_snapshots")
#     .select("*") # *로 먼저 확인한 후 요소별로 조회 ㅇㅇ
#     .execute()
# )
# rprint(response)

# MCP Agent

In [25]:
from rich import print as rprint
from dotenv import load_dotenv

load_dotenv()

True

### 1. 질문 생성

In [69]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

class Query(BaseModel):
    """
    뷰티 제품의 소구점을 다양한 관점으로 파악해서 RAG용 질문 리스트
    - 최소 5개 ~ 최대 10개
    """
    query: list[str] = Field(
        min_length=5,
        max_length=10,
        description="서로 다른 관점의 짧은 질문"
    )

system_prompt = """
너는 브랜드의 인플루언서 매칭을 돕는 마케팅 전략가다.
입력된 제품 설명에서 '소구점(USP)'을 뽑아 Pinecone RAG 검색에 바로 쓸 수 있는 '검색 쿼리 문자열' 리스트를 만든다.

# 중요: '질문'을 쓰지 마라. 검색창에 붙여넣는 짧은 구문만 출력해라.

### 출력 규칙:
- 5~10개
- 각 항목은 3~8개의 핵심 키워드 또는 짧은 구문(1~2 줄).
- 문장부호(?, ., !), 존댓말/서술어(합니다/하세요/인가요), 의문사(무엇/어떻게/왜) 금지
- 가능한 경우 동의어/표현변형을 함께 포함(예: 트러블=여드름, 톤업=미백)
- 너무 일반적인 단어만으로 끝내지 말고 제품 특징(성분/기술/효능/타깃/사용상황) 중 최소 2개를 포함

### 작성 예시
- 나쁜 예: "화해 글로우픽 어워드 1위 글로벌 K뷰티 검증 블랙헤드 베스트셀러(소구점이 아닌, 성과)", "압도적 누적 판매량 1위"
- 좋은 예: "극건성 피부 고보습 크림", "속당김 해결 겨울 스킨케어", "히알루론산 물광 앰플"
"""

query_agent = create_agent(
    # "gpt-5-mini",
    # "google_genai:gemini-3-flash-preview",
    ChatGoogleGenerativeAI(
        model="gemini-3-flash-preview",
        thinking_level="high",
        include_thoughts=True,
        # temperature=1,
        # max_retries=3,
    ),
    system_prompt=system_prompt,
    response_format=ToolStrategy(Query)
)

In [70]:
product = """
[제품 분석 보고서] 원데이즈유 피지쏙쏙 노 모어 블랙헤드
1. 컨셉 및 소구점 (Marketing Concept & USP) - [심층 분석]

이 제품은 **'피지 제거 방식의 패러다임 전환'**을 통해 시장을 장악했습니다. 기존의 물리적 제거 방식이 주는 '쾌감'과 '죄책감(자극)' 사이의 딜레마를 정확히 파고들었습니다.

A. 타겟 고민의 근원과 문제의 본질 (Deep Pain Point)

표면적 문제: 딸기코, 거뭇거뭇한 블랙헤드로 인한 미관상의 스트레스, 화장이 뜨는 현상.

심층적 고통 (Psychological Pain): 소비자는 코팩(Peel-off pack)이나 압출이 모공을 넓히고 피부를 자극한다는 것을 알면서도, 즉각적인 제거를 위해 **'피부 학대'**에 가까운 행위를 반복해 왔습니다. 이 과정에서 **"뽑아내는 쾌감"**과 "모공이 늘어날 것이라는 공포" 사이에서 끊임없이 갈등합니다.

제품이 정의한 문제의 본질: 블랙헤드는 '뽑아야 할 대상'이 아니라 **'단단하게 굳은 기름(결석)'**이므로, 이를 유연하게 만들어 자연스럽게 배출해야 한다는 **'물성(Physical Property)의 변화'**로 접근했습니다.

B. 제품이 제시하는 이상향 (Ideal State & Brand Philosophy)

이상향 정의: "자극 없는 완벽함." 모공을 물리적으로 잡아당기지 않고도 깨끗해질 수 있다는 **'안심감(Peace of Mind)'**을 팝니다. 소비자는 이 제품을 사용함으로써 자신의 피부를 소중히 다루면서도 전문가적인 관리(스케일링)를 하고 있다는 **'스마트한 홈케어족'**의 자아를 갖게 됩니다.

라이프스타일 제안: "뽑지 말고 녹이세요"라는 슬로건은 단순한 사용법 안내가 아닙니다. 이는 자극적인 인스턴트식 해결책을 거부하고, 근본적이고 순한 처방을 선택하는 **'슬로우 뷰티(Slow Beauty)의 미학'**을 제안합니다.

C. 차별화 전략의 디테일 (Differentiation)

'원조(Original)'라는 권위: 상세페이지 상단에서 가장 강조하는 키워드는 "원조", "350만 병 판매", **"압도적 1등"**입니다. 이는 미투(Me-too) 제품이 난무하는 시장에서 "우리가 기준이다"라는 메시지를 던져, 소비자의 **'선택 실패에 대한 두려움'**을 원천 차단합니다.

비유의 힘 (Metaphor): **'#모공 스케일링'**이라는 단어를 사용했습니다. 치과 스케일링처럼 전문가가 굳은 치석을 녹여내듯, 집에서도 전문적인 관리가 가능하다는 인식을 심어줍니다. 또한 '말랑한 피지' vs '딱딱한 피지 결석'을 대비시켜, 제품이 피지를 '액체화' 시킨다는 메커니즘을 시각적으로 설득합니다.

D. 타겟 고객의 언어와 감성 코드

언어 전략: "속 시원하게 녹여버리는", "쏙쏙", "뿌리까지" 등의 단어는 한국 소비자가 피지 제거 제품에서 기대하는 **'카타르시스(쾌감)'**를 자극합니다. 동시에 "순하지만 강력하게", "자극 0.00" 등의 단어로 **'안전성'**을 보장하여 구매 장벽을 낮춥니다.

공감 포인트: "당신의 피지는 어느 쪽입니까?"라는 질문과 극사실적인 코 사진은 소비자가 자신의 코를 거울로 확인하게 만들며 즉각적인 **자기 투영(Self-Projection)**을 유도합니다.

2. 포뮬러 및 기술력 (Formula & Technology)

단순히 피지를 녹이는 것을 넘어, 자극을 최소화하고 후처리를 고려한 '배합의 묘'가 돋보입니다.

성분 배합의 의도:

버지니아풍년화수(Witch Hazel): 전성분 두 번째에 위치할 만큼 고함량입니다. 이는 단순 정제수 베이스가 아님을 의미하며, 피지 제거 후 늘어질 수 있는 **모공 수렴(Astringent)**을 최우선으로 고려했음을 보여줍니다.

식물성 추출물 복합체: 레몬밤, 페퍼민트, 겨우살이 등 30가지 천연 유래 추출물을 사용하여 '화학적 용해'가 아닌 **'식물성 연화'**라는 이미지를 구축했습니다.

특허 성분 (Natural Protector, MultiEx BSASM): 피지를 녹이는 과정은 필연적으로 피부에 부담을 줄 수 있습니다. 이를 상쇄하기 위해 진정 특허 성분을 포함하여 **"강력하지만 자극적이지 않다"**는 모순을 기술적으로 해결했습니다.

제형 (Texture):

워터 타입: 끈적임 없는 맑은 물 제형은 모공 깊숙이 침투하기 유리하며, 화장솜에 적셔 팩처럼 사용하기 최적화된 물성입니다. 이는 오일 제형의 무거움이나 밤 제형의 번거로움을 싫어하는 지성 피부 타겟에게 적합합니다.

3. 효능 입증 및 신뢰도 (Efficacy & Social Proof)

'감성'으로 접근하고 '데이터'로 확신을 주는 전형적인 성공 공식을 따르고 있습니다.

숫자의 위력: **"350만 병 판매"**라는 수치는 그 어떤 임상 데이터보다 강력한 '군중 심리'를 자극합니다. "남들이 다 쓰는 데는 이유가 있다"는 사회적 증거(Social Proof)입니다.

시각적 증명 (Visual Shock):

임상 결과인 **"블랙헤드 면적 개선 69.20%"**를 그래프와 함께 제시하여 논리적 근거를 마련했습니다.

특히, 코 위로 하얗게 불어 올라온 피지 사진은 혐오스러움(Gross)과 쾌감(Satisfying)의 경계에서 시선을 강탈하며, **"내 코에서도 저런 피지가 나올 것"**이라는 강력한 기대감을 심어줍니다.

권위 활용: 화해, 글로우픽, 아마존 1위 등 국내외 뷰티 어워드 8관왕 내역을 "훈장"처럼 나열하여 제품력에 대한 의심을 차단했습니다.

4. 사용 경험 및 루틴 (UX & Ritual)

이 제품은 단순 도포가 아니라 하나의 **'의식(Ritual)'**을 판매합니다.

3단계 리무브 시스템:

불리기 (15-20분): 이 기다림의 시간은 제품이 침투하는 시간이자, 소비자가 '관리받고 있다'는 느낌을 받는 시간입니다.

닦아내기: 면봉이나 패드로 불어난 피지를 걷어내는 물리적 행위를 포함시켰습니다. 이는 화학적 제거만으로는 부족한 '시각적 확인 욕구'를 충족시켜 줍니다. (내 눈으로 닦여 나오는 노란 피지를 확인해야 만족하는 심리)

진정: 모공 수렴으로 마무리하여 죄책감을 덜어줍니다.

디테일: 전용 솜(이중 패드) 사용을 권장하거나 펌핑 용기를 사용하는 등, 사용자의 편의성과 위생을 고려한 UX가 설계되어 있습니다.

5. 마케팅 전략 및 타겟 (Targeting & Psychology)

타겟 페르소나:

"코팩 유목민": 뜯어내는 코팩, 오일, 스크럽 다 써봤지만 효과를 못 봤거나 자극 때문에 중단한 2030 남녀.

"성분 분석가": 화해 앱 랭킹을 신뢰하고, 피부 자극에 민감하게 반응하는 스마트 컨슈머.

"K-뷰티 신뢰층": 일본, 미국 등 글로벌 시장에서의 성공 여부를 제품력의 척도로 삼는 소비자.

공포 vs 이상 전략의 혼합:

초반부에는 "모공이 늘어날 수 있다"는 공포를, 중반부에는 "말랑하게 녹여내는 이상적인 방법"을 제시하는 Problem-Solution 구조가 명확합니다.

글로벌 확장성: 상세페이지 후반부에 일본, 중국, 미국 등 글로벌 진출 현황을 보여주는 것은 국내 소비자에게도 **"세계가 인정한 제품"**이라는 역수입된 신뢰(Reverse Authority)를 형성합니다.

[총평 및 인사이트]

원데이즈유 피지쏙쏙은 단순히 피지를 없애는 제품이 아니라, 피지 제거에 대한 소비자의 '죄책감(자극)'을 없애주는 제품입니다.

이 제품의 성공 요인은 **"뽑지 말고 녹이라"**는 직관적인 카피를 통해, 기존의 고통스러운 제거 방식에 지친 소비자들에게 **'순하고 과학적인 대안'**을 제시했다는 점입니다. 상세페이지는 '원조'로서의 자신감과 압도적인 판매량(350만 병)을 지속적으로 노출하며, 경쟁사들이 따라올 수 없는 '시장 지배자(Market Leader)'의 프레임을 견고하게 구축하고 있습니다.

특히 15분간 피지를 불린 후 **'닦아내는 행위'**를 통해, 자극은 줄이되 피지가 제거되는 시각적 쾌감은 그대로 유지한 점이 UX 측면에서의 핵심 성공 요인으로 분석됩니다.
"""

In [71]:
result = query_agent.invoke({
    "messages": [{
        "role": "user",
        "content": product
    }]
})

In [ ]:
rprint(result["structured_response"])

Query(
    query=[
        '저자극 피지 연화제 블랙헤드 녹이는 방식 식물 성분 배합',
        '코팩 자극 모공 늘어남 방지 민감성 피부 블랙헤드 케어',
        '버지니아풍년화수 위치하젤 함유 모공 수렴 수축 워터 타입 피지 제거제',
        '홈케어 모공 스케일링 피지 불리기 딥클렌징 화이트헤드 녹이기',
        '딸기코 화이트헤드 개선 딱딱한 피지 결석 연화 배출 기술',
        '원조 피지 쏙쏙 350만 병 판매 블랙헤드 제거제 베스트셀러 세정력',
        '지성 피부 T존 유분 관리 블랙헤드 제거 액체 팩 모공 스케일링',
        '30가지 식물 추출물 저자극 피지 연화 모공 진정 특허 성분 케어'
    ]
)

In [83]:
result["structured_response"].query

['저자극 피지 연화제 블랙헤드 녹이는 방식 식물 성분 배합',
 '코팩 자극 모공 늘어남 방지 민감성 피부 블랙헤드 케어',
 '버지니아풍년화수 위치하젤 함유 모공 수렴 수축 워터 타입 피지 제거제',
 '홈케어 모공 스케일링 피지 불리기 딥클렌징 화이트헤드 녹이기',
 '딸기코 화이트헤드 개선 딱딱한 피지 결석 연화 배출 기술',
 '원조 피지 쏙쏙 350만 병 판매 블랙헤드 제거제 베스트셀러 세정력',
 '지성 피부 T존 유분 관리 블랙헤드 제거 액체 팩 모공 스케일링',
 '30가지 식물 추출물 저자극 피지 연화 모공 진정 특허 성분 케어']

In [ ]:
# gpt
Query(
    query=[
        '피지 결석 녹이기, 워터 제형, 모공 스케일링, 15분 불리기, 닦아내기',
        '버지니아풍년화수 고함량, 모공 수렴, 지성 피부 타깃, 순한 진정 성분',
        '식물성 추출물 30종, 식물성 연화, 민감 피부 사용, 자극 0.00',
        '특허 성분 Natural Protector MultiEx BSASM, 진정 효능, 블랙헤드 개선 69.2퍼센트 임상',
        '원조 브랜드, 350만 병 판매, 시장 지배력, 구매 신뢰',
        '뽑지 말고 녹이기, 슬로우 뷰티, 홈케어 스케일링, 의식형 루틴 3단계',
        '시각적 쾌감, 불어난 피지 닦아내기, UX 만족감, 코팩 유목민 타겟',
        '워터 타입 침투력, 패드 팩 최적화, 지성용 산뜻 텍스처, 위생적 펌핑'
    ]
)

## 2. RAG search

In [73]:
from dotenv import load_dotenv
import os

from langchain_openai import OpenAIEmbeddings

load_dotenv()

INDEX_NAME = "influencer-rag"
EMBED_DIM = 1536


# Initiallize Embedding model
embeddings = OpenAIEmbeddings(
	model="text-embedding-3-small"
)

In [74]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone Client
pc = Pinecone()

# Check index nor Create ``
if INDEX_NAME not in pc.list_indexes().names():
	pc.create_index(
		name=INDEX_NAME,
		dimension=EMBED_DIM,
		metric="cosine",
		spec=ServerlessSpec(
			cloud="aws",
			region="us-east-1"
		)
	)

index = pc.Index(INDEX_NAME)

/home/jun99/project-archeive/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [75]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(
    index=index, 
    embedding=embeddings
)

In [ ]:
# similarity_search, similarity_search_with_score
results = vector_store.similarity_search(
    "식물성 추출물 30종, 식물성 연화, 민감 피부 사용, 자극 0.00",
    k=10,
    # filter={"source": "..."}
)
for res, score in results:
    # print(res.metadata["influencer"]) # similarity_search
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.509953] [3] 스펙 최적화 (Spec Optimization):
고농축 병풀추출물과 판테놀 성분: 이 두 가지 핵심 성분은 피부 진정과 장벽 강화라는 **‘확실한 효능’**을 제공하여 ‘문제 해결’이라는 데이터적 요구사항을 충족시킨다.
저자극·단순 성분 포뮬러: ‘논-스트레스 포뮬러’는 민감성 피부도 매일 부담 없이 사용할 수 있게 하여, ‘새우깡템’의 조건인 **‘손이 자주 가는 실용성’**을 기능적으로 구현한다. ‘피부과 추천’, ‘저자극 임상 완료’ 등의 마크는 솔아의 ‘신뢰’ 페르소나를 더욱 강화한다. [{'influencer': 'SOL_A', 'sns': 'youtube', 'source': 'data/youtube/SOL_A.txt'}]
* [SIM=0.464680] 예를 들어, “요즘 꿀템 딱 두 가지만 추천! (만능 스펀지, 유분잡는 컨실러?)” (MaZTD_AeAcc) 콘텐츠는 ‘유분’이라는 명확한 문제를 제시하고 해결책을 보여주어 높은 호응을 얻었다. 네이밍 쿠션 영상 (SICg_brt_1Q)에서는 쿠션 사용 전후의 피부 커버력 차이를 명확한 {전후} 대비 사진으로 제시했으며, 클리오 펜슬 라이너 리뷰 (yVl_Q1UELBo)에서는 물을 묻히고 문지르는 **{테스트}**를 통해 워터프루프 기능을 객관적으로 증명했다. ‘새우깡템’은 바로 이러한 ‘문제 해결 능력’이 검증된 제품이기에, 데이터 기반 성공 공식에 완벽히 부합한다.
[3] 핵심 아이템 및 선정 근거 (Key Items & Strategic Fit)
Item: 코어 디펜스 시카 세럼 (Core Defense Cica Serum)
[1] 맥락적 필연성 (Contextual Necessity): 솔아와 같이 수많은 제품을 테스트하는 뷰티 인플루언서에게 가장 근본적인 결핍은 ‘무너진 피부 기초 체력’과 ‘잦은 자극으로 인한 민감성’이다. 화려한 기능성 제품 이전에, 어떤 제품과도 충돌 없이 피부의 기본을 단단하게 잡아줄 ‘기본템’이자 ‘방어템’이 절실하게 필요하다.

### 병렬 처리

In [85]:
retreiver = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

queries= result["structured_response"].query


results = retreiver.batch(
    queries,
    config={"max_concurrency": len(queries)}
)

In [86]:
# 결과: 각 쿼리별 Document 객체 리스트
for query, docs in zip(queries, results):
    print(f"Query: {query}")
    print(f"Retrieved {len(docs)} documents")

Query: 저자극 피지 연화제 블랙헤드 녹이는 방식 식물 성분 배합
Retrieved 3 documents
Query: 코팩 자극 모공 늘어남 방지 민감성 피부 블랙헤드 케어
Retrieved 3 documents
Query: 버지니아풍년화수 위치하젤 함유 모공 수렴 수축 워터 타입 피지 제거제
Retrieved 3 documents
Query: 홈케어 모공 스케일링 피지 불리기 딥클렌징 화이트헤드 녹이기
Retrieved 3 documents
Query: 딸기코 화이트헤드 개선 딱딱한 피지 결석 연화 배출 기술
Retrieved 3 documents
Query: 원조 피지 쏙쏙 350만 병 판매 블랙헤드 제거제 베스트셀러 세정력
Retrieved 3 documents
Query: 지성 피부 T존 유분 관리 블랙헤드 제거 액체 팩 모공 스케일링
Retrieved 3 documents
Query: 30가지 식물 추출물 저자극 피지 연화 모공 진정 특허 성분 케어
Retrieved 3 documents
